In [1]:
# !apt-get update ; apt-get install -y graphviz libgraphviz-dev
# %pip install scikit_learn
# %pip install pydot
# %pip install pydotplus
# %pip install graphviz
import pandas as pd
import os
import glob
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import mean_squared_error
from tensorflow import keras
import tensorflow as tf
import random
from datetime import datetime
from tensorflow.keras.models import load_model

%matplotlib inline
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1" # do not use GPU
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))
print("Tensorflow version:", tf.__version__)

physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    
# Attempt to make runs more reproducible
seed_value=20212042
print("Using seed value: %d" % seed_value)
os.environ['PYTHONHASHSEED']=str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value) # tensorflow 2.x

# Set paths
PRIZE_DATA = False
base_dir = "/workspace/C683_Assignment"
data_dir = os.path.join(base_dir, "prize-data" if PRIZE_DATA else "ml-100k")
movie_titles = os.path.join(data_dir, "movie_titles.csv")
model_filepath_tmpl = os.path.join(base_dir, "model-weights", "weights-%s-%s-%d-rmse%.04f-batch%d.h5")
all_data_filepath  = os.path.join(data_dir, "all_data.csv")
train_data_filepath  = os.path.join(data_dir, "train_data.pkl")
test_data_filepath = os.path.join(data_dir, "test_data.pkl")

# Utility classes / functions
def get_genres(titles):
    return [col for col in titles if col.startswith("genre.")]

class TrainTestSequence(keras.utils.Sequence):
    def __init__(self, data_set, titles, batch_size):
        self.data_set = data_set
        self.titles = titles
        self.genre_cols = get_genres(titles)
        self.batch_size = batch_size
        
    def __len__(self):
        return self.data_set.shape[0] // self.batch_size
    
    def __getitem__(self, idx):
        batch = self.data_set[idx*self.batch_size:(idx+1)*self.batch_size]
        batch = batch.merge(titles, how="inner", left_on="movie", right_on="id")
        return [np.array(batch["user_ord"]), np.array(batch["movie_ord"]), np.array(batch[self.genre_cols]), np.array(batch["year"])], \
                np.array(batch["rating"])
    
def make_generator(data_set, titles, batch_size, worker_count, use_multiprocessing):
    seq = TrainTestSequence(data_set, titles, batch_size)
    enq = keras.utils.OrderedEnqueuer(seq, use_multiprocessing=use_multiprocessing)
    enq.start(workers=worker_count, max_queue_size=worker_count*20)
    return enq

Num GPUs Available: 1
Tensorflow version: 2.4.0
Using seed value: 20212042


In [2]:
# Constants
USER_COUNT = 480189 if PRIZE_DATA else 943
MOVIE_COUNT = 17770 if PRIZE_DATA else 1682
TRAIN_BATCH_SIZE = 512 if PRIZE_DATA else 32
TEST_BATCH_SIZE = 16384 if PRIZE_DATA else 256
MAX_YEAR = 2021
TRAIN_ITERATIONS_COUNT = 20
print("TRAIN_BATCH_SIZE: %d" % TRAIN_BATCH_SIZE)
print("TEST_BATCH_SIZE: %d" % TEST_BATCH_SIZE)
print("TRAIN_ITERATIONS_COUNT: %d" % TRAIN_ITERATIONS_COUNT)

# Prepare train/test if needed
if not os.path.isfile(train_data_filepath) or not os.path.isfile(test_data_filepath):
    all_data = None
    if PRIZE_DATA:
        print("Loading Netflix Prize data set")
        all_data = pd.read_csv(all_data_filepath,
                               names=[ "movie", "user", "rating", "date" ],
                               dtype={ "movie": np.int, "user": np.int, "movie": np.int },
                               parse_dates=["date"])
    else:
        all_data = pd.read_csv(all_data_filepath,
                               names=[ "user", "movie", "rating", "date" ],
                               dtype={ "movie": np.int, "user": np.int, "movie": np.int },
                               parse_dates=["date"],
                               sep="\t")
    print("Creating ordinal encodings for movie and user")
    ord_enc = OrdinalEncoder(dtype=np.int)
    all_data["movie_ord"] = ord_enc.fit_transform(all_data[["movie"]])
    all_data["user_ord"] = ord_enc.fit_transform(all_data[["user"]])
    print(all_data.head())
    print("Splitting into train and test data")
    train_data, test_data = train_test_split(all_data, test_size=0.15)
    print("Writing train data to a file")
    train_data.to_pickle(train_data_filepath)
    print("Writing test data to a file")
    test_data.to_pickle(test_data_filepath)
else:
    print("Loading Prize train data")
    train_data = pd.read_pickle(train_data_filepath)
    print("Loading Prize test data")
    test_data = pd.read_pickle(test_data_filepath)
    
display(train_data.head())
train_batch_count = train_data.shape[0] // TRAIN_BATCH_SIZE
steps_per_epoch = train_batch_count//(1000 if PRIZE_DATA else 20)
display(test_data.head())
test_batch_count = test_data.shape[0] // TEST_BATCH_SIZE

print("Loading movie titles")
titles = None
if PRIZE_DATA:
    titles = pd.read_csv(movie_titles)
else:
    titles = pd.read_csv(movie_titles,
                   names=[ "id", "title", "release", "video_release", "url",
                          "genre.unknown", "genre.Action", "genre.Adventure", "genre.Animation", "genre.Children's",
                          "genre.Comedy", "genre.Crime", "genre.Documentary", "genre.Drama", "genre.Fantasy",
                          "genre.Film-Noir", "genre.Horror", "genre.Musical", "genre.Mystery", "genre.Romance", "genre.Sci-Fi",
                          "genre.Thriller", "genre.War", "genre.Western" ],
                   parse_dates=["release"],
                   sep="|")
    titles = titles.dropna(subset=["release"])
    titles["year"] = titles["release"].apply(lambda x: int(x.year))
display(titles.head())

TRAIN_BATCH_SIZE: 32
TEST_BATCH_SIZE: 256
TRAIN_ITERATIONS_COUNT: 20
Loading Prize train data
Loading Prize test data


user  movie  rating       date  movie_ord  user_ord
23922   491     45       5  891189631         44       490
79788     7     11       3  891352451         10         6
77918   795    319       4  880554132        318       794
58824   727    386       2  883712805        385       726
80907   921    215       4  879380677        214       920

user  movie  rating       date  movie_ord  user_ord
77336   593    417       5  875671598        416       592
3583    328    742       4  885047309        741       327
71568     6    468       3  883602174        467         5
39070   286    132       5  877531791        131       285
58221   303    477       3  879483827        476       302

Loading movie titles


id              title    release  video_release  \
0   1   Toy Story (1995) 1995-01-01            NaN   
1   2   GoldenEye (1995) 1995-01-01            NaN   
2   3  Four Rooms (1995) 1995-01-01            NaN   
3   4  Get Shorty (1995) 1995-01-01            NaN   
4   5     Copycat (1995) 1995-01-01            NaN   

                                                 url  genre.unknown  \
0  http://us.imdb.com/M/title-exact?Toy%20Story%2...              0   
1  http://us.imdb.com/M/title-exact?GoldenEye%20(...              0   
2  http://us.imdb.com/M/title-exact?Four%20Rooms%...              0   
3  http://us.imdb.com/M/title-exact?Get%20Shorty%...              0   
4  http://us.imdb.com/M/title-exact?Copycat%20(1995)              0   

   genre.Action  genre.Adventure  genre.Animation  genre.Children's  ...  \
0             0                0                1                 1  ...   
1             1                1                0                 0  ...   
2             0                0                0                 0  ...   
3             1                0                0                 0  ...   
4             0                0                0                 0  ...   

   genre.Film-Noir  genre.Horror  genre.Musical  genre.Mystery  genre.Romance  \
0                0             0              0              0              0   
1                0             0              0              0              0   
2                0             0              0              0              0   
3                0             0              0              0              0   
4                0             0              0              0              0   

   genre.Sci-Fi  genre.Thriller  genre.War  genre.Western  year  
0             0               0          0              0  1995  
1             0               1          0              0  1995  
2             0               1          0              0  1995  
3             0               0          0              0  1995  
4             0               1          0              0  1995  

[5 rows x 25 columns]

In [3]:
# Define model
user_input = keras.layers.Input(shape=(1,), name="User_ID")
user_embedding = keras.layers.Embedding(USER_COUNT, 96, name="User_Embedding_50")(user_input)
user_embedding = keras.layers.Flatten(name="User_Row_Vector")(user_embedding)
user_embedding = keras.layers.Dropout(0.4, name="User_Dropout_0.4")(user_embedding)

movie_input = keras.layers.Input(shape=(1,), name="Movie_ID")
movie_embedding = keras.layers.Embedding(MOVIE_COUNT, 96, name="Movie_Embedding_50")(movie_input)
movie_embedding = keras.layers.Flatten(name="Movie_Row_Vector")(movie_embedding)
movie_embedding = keras.layers.Dropout(0.4, name="Movie_Dropout_0.4")(movie_embedding)

genre_count = len(get_genres(titles))
genre_input = keras.layers.Input(shape=(genre_count,), name="Genres_1-Hot")
genre_dense = keras.layers.Dense(32, name="Genres_Dense_8")(genre_input)
genre_dense = keras.layers.Dropout(0.2, name="Genres_Dropout_0.2")(genre_dense)

year_input =  keras.layers.Input(shape=(1,), name="Movie_Year")
year_embedding = keras.layers.Embedding(MAX_YEAR, 32, name="Year_Embedding_8")(year_input)
year_embedding = keras.layers.Flatten(name="Year_Row_Vector")(year_embedding)
year_embedding = keras.layers.Dropout(0.2, name="Year_Dropout_0.2")(year_embedding)

content = keras.layers.Concatenate(name="Concat_AllContent")([genre_dense, year_embedding])
content = keras.layers.Dense(96, name="AllContent_Dense_16")(content)
content = keras.layers.Dropout(0.4, name="AllContent_Dropout_0.4")(content)

nn = keras.layers.Multiply(name="Multiply")([user_embedding, movie_embedding, content])
nn = keras.layers.Dense(128, name="Dense_128")(nn)
nn = keras.layers.Dropout(0.6, name="Dropout_0.6")(nn)
nn = keras.layers.Dense(64, name="Dense_64")(nn)
nn = keras.layers.Dropout(0.4, name="Dropout_0.4")(nn)
nn = keras.layers.Dense(1, name="Dense_1_Regression")(nn)
model = keras.models.Model([user_input, movie_input, genre_input, year_input], nn)
print(model.summary())
display(keras.utils.plot_model(model))

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Movie_Year (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
Genres_1-Hot (InputLayer)       [(None, 19)]         0                                            
__________________________________________________________________________________________________
Year_Embedding_8 (Embedding)    (None, 1, 32)        64672       Movie_Year[0][0]                 
__________________________________________________________________________________________________
Genres_Dense_8 (Dense)          (None, 32)           640         Genres_1-Hot[0][0]               
______________________________________________________________________________________________

In [4]:
# Train model
optimizer = keras.optimizers.Adam(learning_rate=0.005 if PRIZE_DATA else 0.001)
model.compile(optimizer=optimizer, loss='mse', metrics=keras.metrics.RootMeanSquaredError())
train_generator = make_generator(train_data, titles, TRAIN_BATCH_SIZE, 6, True).get()
evaluate_generator = make_generator(test_data, titles, TEST_BATCH_SIZE, 6, True)
for i in range(TRAIN_ITERATIONS_COUNT):
    model.fit(train_generator,
              epochs=1, steps_per_epoch=steps_per_epoch,
              verbose=1)

    # Evaluate and model
    y_pred = model.predict(evaluate_generator.get(), steps=test_batch_count, verbose=1)
    evaluate_generator.stop()
    evaluate_generator.start(workers=6, max_queue_size=1)
    rmse = mean_squared_error(test_data[:y_pred.shape[0]]["rating"], y_pred)
    print("RMSE: %.04f" % rmse)
    
model_filepath = model_filepath_tmpl % ("prize" if PRIZE_DATA else "ml100k", datetime.now().strftime("%d_%m_%Y_%H_%M"), 0, rmse, TRAIN_BATCH_SIZE)
model.save(model_filepath)

58/58 [==============================] - 0s 4ms/step
RMSE: 1.2980
58/58 [==============================] - 0s 5ms/step
RMSE: 1.2804
58/58 [==============================] - 0s 4ms/step
RMSE: 1.2804WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.
RMSE: 1.2898
58/58 [==============================] - 0s 5ms/step
RMSE: 1.3220
58/58 [==============================] - 0s 5ms/step
RMSE: 1.3220
RMSE: 1.2713
58/58 [==============================] - 0s 5ms/step
RMSE: 1.3472
58/58 [==============================] - 0s 5ms/step
RMSE: 1.3472
RMSE: 1.2916
58/58 [==============================] - 0s 5ms/step
RMSE: 1.2916
RMSE: 1.2900
58/58 [==============================] - 0s 6ms/step
RMSE: 1.2900
RMSE: 1.2786
58/58 [==============================] - 0s 5ms/step
RMSE: 1.2701
58/58 [==============================] - 0s 6ms/step
RMSE: 1.2701
RMSE: 1.3107
58/58 [=======================

58/58 [==============================] - 0s 5ms/step

RMSE: 1.2844
58/58 [==============================] - 0s 6ms/step
RMSE: 1.2912
58/58 [==============================] - 0s 5ms/step
RMSE: 1.2716
58/58 [==============================] - 0s 6ms/step

RMSE: 1.2881
58/58 [==============================] - 0s 6ms/step
RMSE: 1.2709
